# Homework 6

In [1]:
!pip install simpy

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import random
import simpy

# Question 13.2
In this problem you, can simulate a simplified airport security system at a busy airport. Passengers arrive according to a Poisson distribution with λ1 = 5 per minute (i.e., mean interarrival rate μ1 = 0.2 minutes) to the ID/boarding-pass check queue, where there are several servers who each have exponential service time with mean rate μ2 = 0.75 minutes. [Hint: model them as one block that has more than one resource.] After that, the passengers are assigned to the shortest of the several personal-check queues, where they go through the personal scanner (time is uniformly distributed between 0.5 minutes and 1 minute).

Use the Arena software (PC users) or Python with SimPy (PC or Mac users) to build a simulation of the system, and then vary the number of ID/boarding-pass checkers and personal-check queues to determine how many are needed to keep average wait times below 15 minutes. [If you’re using SimPy, or if you have access to a non-student version of Arena, you can use λ1 = 50 to simulate a busier airport.]

In [3]:
#servers = number of ID/boarding-pass check queue servers
#scanners = number of personal scanners
servers = 2
num_scanners = 4
r_seed = 500

#mean rates
arrival_rate = 50 #passengers per minute
service_rate = .75 #passengers served per minute
min_scan = .5 #personal scanner minimum time to get through; uniformly distributed with max time below
max_scan = 1.0 # #personal scanner maximum time to get through;

# define simulation timing
run_time = 150 # minutes per simulation
reps = 50 # number of simulation replications

#variables we will store answers in later
serviceWait = 0
scanWait = 0
sysTime = 0
timeWait = 0
timeServer = 0
timeServiceComplete = 0
timeScanner = 0
timeScannerComplete = 0
passengers = 0

In [4]:
#Creating the simulation
class Airport(object):

    #define both the servers and personal scanners
    def __init__(self, env):

        self.env = env

        #set up # of servers
        self.server = simpy.Resource(env, servers)

        #set up # of scanners
        self.scanners = []

        #assign resource(s) to each scanner
        for i in range(0,25):
            resource = simpy.Resource(env, capacity = 1)
            self.scanners.append(resource)

    #how long a passenger takes at the service desk
    def check(self, passenger):
        yield self.env.timeout(random.expovariate(1/service_rate))

    #how long a passenger takes to scan themselves in
    def scan(self, passenger):
        yield self.env.timeout(random.uniform(max_scan , min_scan))

#give attributes to each passenger as to how they move through the servers and personal scanners
def passenger(env, name, s):

    global serviceWait
    global scanWait
    global sysTime
    global timeWait
    global timeServer
    global timeServiceComplete
    global timeScanner
    global timeScannerComplete
    global passengers

    #time passenger arrives
    arrivaltime = env.now
    print('%s arrives at time %.2f' % (name, arrivaltime))

    with s.server.request() as request:
        yield request
        print('check queue length = %d' % len(s.server.queue))

        #the time the passenger arrives at a server
        timeServer = env.now
        print('%s gets to server at time %.2f' % (name, timeServer))

        yield env.process(s.check(name))

        # time passenger completes server
        timeServiceComplete = env.now
        print('%s complete server at time %.2f' % (name, timeServiceComplete))

    min_q = 0

    with s.scanners[min_q].request() as request:
        yield request
        print('scanner queue length = %d' % len(s.scanners[min_q].queue))

        for i in range(1, num_scanners):
            if (len(s.scanners[i].queue) < len(s.scanners[min_q].queue)):
                min_q = i

        # time passenger arrives at scanner
        timeScanner = env.now
        print('%s gets to scanner at time %.2f' % (name, timeScanner))

        yield env.process(s.scan(name))

        timeScannerComplete = env.now
        print('%s complete scanner at time %.2f' % (name, timeScannerComplete))

    #time at the end of simulation
    timeExit = env.now
    print('%s gets to complete system time %.2f' % (name, timeExit))

    sysTime = sysTime + (timeExit - arrivaltime)
    serviceWait = serviceWait + (timeServer - arrivaltime)
    scanWait = scanWait + (timeScannerComplete - timeServiceComplete)
    timeWait = (serviceWait + scanWait)
    
def setup(env):

    airport = Airport(env)

    i = 0

    while True:
        yield env.timeout(random.expovariate(1.0/service_rate))
        i += 1
        env.process(passenger(env, 'Passenger %d' % i, airport))

In [5]:
#variables we'll store the wait times in
avg_wait_time = []
avg_check_time = []
avg_scan_time = []
avg_sys_time = []

for i in range(0, reps):

    env = simpy.Environment()
    env.process(setup(env))
    env.run(until = run_time)

    avg_wait_time.append(timeWait/125)
    avg_check_time.append(serviceWait/125)
    avg_scan_time.append(scanWait/125)
    avg_sys_time.append(sysTime/125)

    passengers = 0
    sysTime = 0
    serviceWait = 0
    scanWait = 0
    timeWait = 0

sim_wait_avg = sum(avg_wait_time) / reps
sim_check_avg = sum(avg_check_time) / reps
sim_scan_avg = sum(avg_scan_time) / reps
sim_sys_time = sum(avg_sys_time) / reps

print('Average cumulative wait time: ' + str(sim_wait_avg))
print('Average cumulative service time: ' + str(sim_check_avg))
print('Average cumulative scanning time: ' + str(sim_scan_avg))
print('Average cumulative system time: ' + str(sim_sys_time))

Passenger 1 arrives at time 1.02
check queue length = 0
Passenger 1 gets to server at time 1.02
Passenger 2 arrives at time 2.11
check queue length = 0
Passenger 2 gets to server at time 2.11
Passenger 3 arrives at time 2.54
Passenger 4 arrives at time 2.64
Passenger 5 arrives at time 2.78
Passenger 6 arrives at time 2.86
Passenger 1 complete server at time 3.19
scanner queue length = 0
Passenger 1 gets to scanner at time 3.19
check queue length = 3
Passenger 3 gets to server at time 3.19
Passenger 7 arrives at time 3.40
Passenger 8 arrives at time 3.41
Passenger 3 complete server at time 3.69
check queue length = 4
Passenger 4 gets to server at time 3.69
Passenger 9 arrives at time 3.84
Passenger 1 complete scanner at time 3.85
Passenger 1 gets to complete system time 3.85
scanner queue length = 0
Passenger 3 gets to scanner at time 3.85
Passenger 2 complete server at time 4.26
check queue length = 4
Passenger 5 gets to server at time 4.26
Passenger 10 arrives at time 4.61
Passenger 4

Since I decided to run this simulation in Python, I figured I might as well attempt to simulate a busier airport with ~50 people arriving per minute. After playing around with number of resources vs personal scanners, I've found that having at least 2 resources to get people checked in and then 3 personal scanners to go through eliminated most bottlenecks and allowed people to get through security in ~11.8 minutes. Considering the large number of people coming in every minute, that's a rather short wait to get into the airport. As a frequent flyer myself, I've done my fair share of waiting in the airport so I'd be curious to see how these numbers were simulated at each airport (staff availability, time of day, time of week, available security equipment, number of flights, etc.).